In [85]:
# %%
import numpy as np
import json
import chempy as chem
from chempy.util import periodic as per
import gen.scriptgen.soilconctomcnp as sm
import gen.scriptgen.miscgen as misc
import pandas as pd
from tqdm import tqdm

from chempy import Substance
import periodictable
import tabulate

In [86]:
periodic_table = {per.atomic_number(sym): sym for sym in per.symbols}
material_names = ['Carbon', 'Water', 'Quartz', 'Feldspar', 'Mica']
material_formulas = ['C', 'H2O', 'SiO2', 'NaAlO2(SiO2)3', 'K2Al2O5(Si2O5)3Al4(OH)4']
substances = [Substance.from_formula(formula) for formula in material_formulas]
compositions = [substance.composition for substance in substances]
# replace the keys of the compositions with the atomic symbols
compositions = [
	{periodic_table[atomic_number]: value for atomic_number, value in composition.items()}
	for composition in compositions
]
mass_fractions = [chem.mass_fractions(composition) for composition in compositions]
material_frame = pd.DataFrame(mass_fractions, index=material_names)
# make nans zero
material_frame = material_frame.fillna(0)
material_frame_percent = material_frame * 100
material_frame_percent.index.name = 'Material'
material_frame_percent.columns = [column + ' %' for column in material_frame_percent.columns]
print(tabulate.tabulate(material_frame_percent, 
    headers='keys', tablefmt='pipe', showindex=True, floatfmt=".1f"))

| Material   |   C % |   H % |   O % |   Si % |   Na % |   Al % |   K % |
|:-----------|------:|------:|------:|-------:|-------:|-------:|------:|
| Carbon     | 100.0 |   0.0 |   0.0 |    0.0 |    0.0 |    0.0 |   0.0 |
| Water      |   0.0 |  11.2 |  88.8 |    0.0 |    0.0 |    0.0 |   0.0 |
| Quartz     |   0.0 |   0.0 |  53.3 |   46.7 |    0.0 |    0.0 |   0.0 |
| Feldspar   |   0.0 |   0.0 |  48.8 |   32.1 |    8.8 |   10.3 |   0.0 |
| Mica       |   0.0 |   0.5 |  48.2 |   21.2 |    0.0 |   20.3 |   9.8 |


In [87]:
elements = material_frame.columns.tolist()
atomic_numbers = [per.atomic_number(element) for element in elements]
mcnp_identifiers = [1000*(atomic_number) for atomic_number in atomic_numbers]

translate_dict = {
    1000: 1001,  # Hydrogen
    8000: 8016,  # Oxygen
    14000: 14028,  # Silicon
    26000: 26054,  # Iron
    6000: 6000,  # Carbon
    12000: 12023,  # Sodium
    11000: 11021,  # Magnesium
    13000: 13027,  # Aluminum
}
mcnp_identifiers = [translate_dict.get(x, x) for x in mcnp_identifiers]
# get the density of each element in g/cm^3
densities = [periodictable.elements.symbol(elem).density for elem in elements]
elemental_frame = pd.DataFrame({
    'MCNP Identifier': mcnp_identifiers,
    # 'Density (g/cm^3)': densities
}, index=elements)
_elemental_frame = elemental_frame.copy()
_elemental_frame.index.name = 'Element'
_elemental_frame.columns.name = 'Element'
print(tabulate.tabulate(_elemental_frame.T, 
      headers='keys', tablefmt='pipe', showindex=True))

| Element         |    C |    H |    O |    Si |    Na |    Al |     K |
|:----------------|-----:|-----:|-----:|------:|------:|------:|------:|
| MCNP Identifier | 6000 | 1001 | 8016 | 14028 | 11021 | 13027 | 19000 |


In [88]:
def soil_characteristic_function(X, character=None, elem_labels=elements):
    n = X.shape[0]
    out = np.zeros((n, len(elem_labels)))
    if character:
        elem_index = elem_labels.index(character)
        out[:, elem_index] = 1
    else:
        pass
    return out
def material_characteristic_function(X, character=None, material_frame=material_frame):
    n = X.shape[0]
    out = np.zeros((n, len(material_frame.columns)))
    if character:
        out[:, :] = material_frame.loc[character].values
    else:
        pass
    return out

def material_to_carbon_mechanical_mix(X, character, carbon_fraction=0, material_frame=material_frame):
    n = X.shape[0]
    out = np.zeros((n, len(material_frame.columns)))
    if character:
        out += material_characteristic_function(X, character, material_frame)* (1 - carbon_fraction)
        out += material_characteristic_function(X, 'Carbon', material_frame) * carbon_fraction
    else:
        pass
    return out

def material_to_water_mechanical_mix(X, character, water_fraction=0, material_frame=material_frame):
    n = X.shape[0]
    out = np.zeros((n, len(material_frame.columns)))
    if character:
        out += material_characteristic_function(X, character, material_frame) * (1 - water_fraction)
        out += material_characteristic_function(X, 'Water', material_frame) * water_fraction
    else:
        pass
    return out

def material_carbon_water_mechanical_mix(X, character=None, carbon_fraction=0, water_fraction=0, material_frame=material_frame):
    n = X.shape[0]
    out = np.zeros((n, len(material_frame.columns)))
    if carbon_fraction < 0 or water_fraction < 0:
        raise ValueError("Carbon and water fractions must be non-negative.")
    if carbon_fraction + water_fraction > 1:
        raise ValueError("Sum of carbon and water fractions must not exceed 1.")
    if character:
        out += material_characteristic_function(X, character, material_frame) * (1 - carbon_fraction - water_fraction)
        out += material_characteristic_function(X, 'Carbon', material_frame) * carbon_fraction
        out += material_characteristic_function(X, 'Water', material_frame) * water_fraction
    else:
        pass
    return out

def density_char(X, density=1.6):
    n = X.shape[0]
    out = np.zeros(n)
    out[:] = density
    return out

In [89]:
densities = np.linspace(1.04, 1.59, 3)

In [90]:
tests = []

In [91]:
# Elemental
for elem in elements:
    test = {}
    test['label'] = f"{elem}"
    test['elem'] = elem
    test['chem_function']  = lambda X, elem=elem: soil_characteristic_function(X, character=elem, elem_labels=elements)
    test['chem_function_name'] = f"soil_characteristic_function_{elem}"
    test['density_function'] = lambda X, density=np.average(densities): density_char(X, density=density)
    test['density_function_name'] = f"density_char_{np.average(densities)}"
    test['resolution'] = (9, 9, 9)
    tests.append(test)

In [92]:
low_carbon_levels = [i/100 for i in range(0, 16)]        # 0.00 .. 0.15
high_carbon_levels = [ (5*i)/100 for i in range(4,21-4)]  # 0.20 .. 1.00
carbon_levels = low_carbon_levels + high_carbon_levels


In [93]:
hydration_levels = [i/100 for i in np.arange(0, 21, 5).tolist()]

In [94]:
hydration_levels

[0.0, 0.05, 0.1, 0.15, 0.2]

In [95]:

test = {}
label = f"Water"
test['label'] = label
test['material'] = 'Water'
test['chem_function'] = lambda X, material='Water': material_carbon_water_mechanical_mix(
    X, character=material, carbon_fraction=0, water_fraction=0, material_frame=material_frame)
test['chem_function_name'] = f"Water"
test['density_function'] = lambda X, density=np.average(densities): density_char(X, density=density)
test['density_function_name'] = f"density_char_{str(np.average(densities)).replace('.', 'p')}"

test['resolution'] = (9, 9, 9)
tests.append(test)
    

In [96]:
# Materials
for material in material_names[2:]:  # Skip Carbon and Water
    for density in densities:
        for carbon_level in carbon_levels:
            for hydration_level in hydration_levels:
                test = {}
                label = f"M{material}_D{str(density).replace('.', 'p')}_C{str(carbon_level).replace('.', 'p')}_H{str(hydration_level).replace('.', 'p')}"
                test['label'] = label
                test['material'] = material
                test['carbon_level'] = carbon_level
                test['hydration_level'] = hydration_level
                # test['chem_function'] = lambda X, material=material, carbon_level=carbon_level: material_to_carbon_mechanical_mix(X, character=material, carbon_fraction=carbon_level, material_frame=material_frame)
                test['chem_function'] = lambda X, material=material, carbon_level=carbon_level, hydration_level=hydration_level: material_carbon_water_mechanical_mix(
                    X, character=material, carbon_fraction=carbon_level, water_fraction=hydration_level, material_frame=material_frame)
                test['chem_function_name'] = f"material_to_carbon_mechanical_mix_{material}_{str(carbon_level).replace('.', 'p')}"
                test['density_function'] = lambda X, density=density: density_char(X, density=density)
                test['density_function_name'] = f"density_char_{str(density).replace('.', 'p')}"

                test['resolution'] = (9, 9, 9)
                tests.append(test)
            

In [97]:
len(tests)

1313

In [98]:
x_pad = 56
y_pad = 45
z_pad= 30

center = (0, 0, 0)

extent = (
    center[0]-x_pad, center[0]+x_pad,
    center[1]-y_pad, center[1]+y_pad,
    center[2]-z_pad, 0,
)

In [99]:
def force_n_digits(x, n):
    # if x is less that 10^n, return 0000...x such that the length is n digits, else return x
    if x < 10**n:
        return f'{x:0{n}d}'
    return f'{x}'

def howmany_digits(x):
    # return the number of digits in x
    if x == 0:
        return 1
    return len(str(x))


In [100]:
tests_df = pd.DataFrame(tests)

In [101]:
tests_df

,label,elem,chem_function,chem_function_name,density_function,density_function_name,resolution,material,carbon_level,hydration_level
0,C,C,<function <lambda> at 0x7410ad6c77f0>,soil_characteristic_function_C,<function <lambda> at 0x74109f0dfd00>,density_char_1.3150000000000002,"(9, 9, 9)",NaN,NaN,NaN
1,H,H,<function <lambda> at 0x74109f0dd2d0>,soil_characteristic_function_H,<function <lambda> at 0x74109f0dd7e0>,density_char_1.3150000000000002,"(9, 9, 9)",NaN,NaN,NaN
2,O,O,<function <lambda> at 0x74109f0df010>,soil_characteristic_function_O,<function <lambda> at 0x74109f0dd630>,density_char_1.3150000000000002,"(9, 9, 9)",NaN,NaN,NaN
3,Si,Si,<function <lambda> at 0x74109f0de950>,soil_characteristic_function_Si,<function <lambda> at 0x74109f0de830>,density_char_1.3150000000000002,"(9, 9, 9)",NaN,NaN,NaN
4,Na,Na,<function <lambda> at 0x74109f0def80>,soil_characteristic_function_Na,<function <lambda> at 0x74109f0dc040>,density_char_1.3150000000000002,"(9, 9, 9)",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1308,MMica_D1p59_C0p8_H0p0,NaN,<function <lambda> at 0x7410965cab90>,material_to_carbon_mechanical_mix_Mica_0p8,<function <lambda> at 0x7410965cac20>,density_char_1p59,"(9, 9, 9)",Mica,0.8,0.00
1309,MMica_D1p59_C0p8_H0p05,NaN,<function <lambda> at 0x7410965cacb0>,material_to_carbon_mechanical_mix_Mica_0p8,<function <lambda> at 0x7410965cad40>,density_char_1p59,"(9, 9, 9)",Mica,0.8,0.05
1310,MMica_D1p59_C0p8_H0p1,NaN,<function <lambda> at 0x7410965cadd0>,material_to_carbon_mechanical_mix_Mica_0p8,<function <lambda> at 0x7410965cae60>,density_char_1p59,"(9, 9, 9)",Mica,0.8,0.10
1311,MMica_D1p59_C0p8_H0p15,NaN,<function <lambda> at 0x7410965caef0>,material_to_carbon_mechanical_mix_Mica_0p8,<function <lambda> at 0x7410965caf80>,density_char_1p59,"(9, 9, 9)",Mica,0.8,0.15


In [102]:
mcnp_identifiers

[6000, 1001, 8016, 14028, 11021, 13027, 19000]

In [103]:
# translate_dict = {
#     1000: 1001,  # Hydrogen
#     8000: 8016,  # Oxygen
#     14000: 14028,  # Silicon
#     26000: 26054,  # Iron
#     6000: 6000,  # Carbon
#     12000: 12023,  # Sodium
#     11000: 11021,  # Magnesium
#     13000: 13027,  # Aluminum
# }
# mcnp_identifiers = [translate_dict.get(x, x) for x in mcnp_identifiers]

In [105]:
sim_folder = "compute/input/"
_id = 0
n_tests = len(tests)
max_digits = howmany_digits(n_tests)
for idx, test in tqdm(tests_df.iterrows(), desc="Generating MCNP scripts", total=n_tests):
    chem_function = test['chem_function']
    density_function = test['density_function']
    res = test['resolution']

    cells, cell_ids, walls, surfaces, mats, avg_sample, midpoints, sides, elems, soil_densities, detector_tallies, detector_tally_ids = sm.make_mcnp(
        test['chem_function'],
        extent,
        test['resolution'],
        mcnp_identifiers,
        density = test['density_function'],
        x_fix=0,
        y_fix=0,
        z_fix=-42,
        z_mul=-1,
        surface_header='200',
        cell_header='9',
        mat_header='40',
        detector_tally_header='8',
        detector_cell='101',
    )

    script = misc.mcnpgen(cells, walls, surfaces, mats, detector_tallies)
    # tests_df.at[idx, 'script'] = script
    
    serial = str(force_n_digits(_id, max_digits))
    _id += 1
    tests_df.at[idx, 'serial'] = serial

    label = test['label']
    filename = f"{label}.txt"
    tests_df.at[idx, 'filename'] = filename

    with open(sim_folder + filename, 'w') as f:
        f.write(script)
    
    soil_info = {
        "cell_ids": cell_ids,
        "walls": walls,
        "sides": sides,
        "midpoints": midpoints,
        "elements": elems,
        "densities": np.array(soil_densities)
    }

    tests_df.at[idx, 'soil_info'] = soil_info


Generating MCNP scripts: 100%|██████████| 1313/1313 [00:16<00:00, 80.81it/s]


In [ ]:

_tests_df = tests_df.copy()

# remove the 'chem_function', 'density_function', and 'hydration_function' columns
_tests_df = _tests_df.drop(columns=['chem_function', 'density_function'])

_tests_df.to_pickle('tests.pkl')
